In [1]:
import optuna
from optuna.integration import BoTorchSampler
import numpy as np

# 最小化したい目的関数の定義
def objective(trial):
    # x, y の範囲を定義
    x = trial.suggest_float("x", -5.0, 5.0)
    y = trial.suggest_float("y", -5.0, 5.0)
    
    # 二次関数 (例えば (x-2)^2 + (y+3)^2 のようなもの)
    return (x - 2) ** 2 + (y + 3) ** 2

# ベイズ最適化を使用して目的関数を最小化
def main():
    # BoTorchSampler を使用
    sampler = BoTorchSampler()
    
    # OptunaのStudyを作成
    study = optuna.create_study(direction="minimize", sampler=sampler)
    
    # 最適化の実行
    study.optimize(objective, n_trials=50)
    
    # 結果の出力
    print("Best Value: ", study.best_value)
    print("Best Parameters: ", study.best_params)

if __name__ == "__main__":
    main()

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_26738/4124746621.py:17: ExperimentalWarning: BoTorchSampler is experimental (supported from v2.4.0). The interface can change in the future.
  sampler = BoTorchSampler()
[I 2024-11-26 14:51:14,923] A new study created in memory with name: no-name-d51325d4-ddf2-43b5-8b3d-243e4fedab81
[I 2024-11-26 14:51:14,925] Trial 0 finished with value: 20.267693934856634 and parameters: {'x': 0.6798774137280894, 'y': 1.3040643921846007}. Best is trial 0 with value: 20.267693934856634.
[I 2024-11-26 14:51:14,925] Trial 1 finished with value: 0.6887817163339843 and parameters: {'x': 1.2281858413213398, 'y': -2.6949022766438}. Best is trial 1 with value: 0.6887817163339843.
[I 2024-11-26 14:51:14,926] Trial 2 finished with value: 20.489817283183474 and parameters: {'x': -0.3551816275411157, 'y': 0.8656094971525841}. Best is trial 1 with value: 0.6887817163339843.
[I 2024-11-26 14:51:14,926] Trial 3 finished with value: 18.580110088434093 and pa

Best Value:  0.0005148785600849143
Best Parameters:  {'x': 1.9858757550311443, 'y': -2.982240938534239}


In [6]:
import torch
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.kernels import MaternKernel, ScaleKernel
from botorch.acquisition.analytic import LogExpectedImprovement
from botorch.optim import optimize_acqf
from botorch.utils.transforms import normalize, unnormalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model


def matern52_logei_candidates_func(
    train_x: torch.Tensor,
    train_obj: torch.Tensor,
    train_con: torch.Tensor | None,
    bounds: torch.Tensor,
    pending_x: torch.Tensor | None,
) -> torch.Tensor:
    """
    カーネルを Matern52 に設定し、Log Expected Improvement (LogEI) を使用した
    Optuna BoTorchSampler のカスタム候補生成関数。
    """

    # 入力の正規化
    train_x = normalize(train_x, bounds=bounds)

    # モデルを Matern52 に変更
    model = SingleTaskGP(
        train_x,
        train_obj,
        covar_module=ScaleKernel(
            MaternKernel(nu=2.5, ard_num_dims=train_x.size(-1))  # ν=2.5 で Matern52
        ),
        outcome_transform=Standardize(m=train_obj.size(-1)),
    )
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    # モデルの学習
    model.train()
    mll.train()
    fit_gpytorch_model(mll)  # モデルに適合

    # LogEI を設定
    best_f = train_obj.max()  # 現在の最良値を取得
    acqf = LogExpectedImprovement(
        model=model,
        best_f=best_f,
    )

    # 標準化された探索空間の境界
    standard_bounds = torch.zeros_like(bounds)
    standard_bounds[1] = 1

    # 獲得関数の最適化
    candidates, _ = optimize_acqf(
        acq_function=acqf,
        bounds=standard_bounds,
        q=1,  # 単一候補の最適化
        num_restarts=10,
        raw_samples=512,
        options={"batch_limit": 5, "maxiter": 200},  # デフォルトの設定
        sequential=True,
    )

    # 元のスケールに戻す
    return unnormalize(candidates.detach(), bounds=bounds)


In [7]:
import optuna
from optuna.integration import BoTorchSampler
import numpy as np

# 最小化したい目的関数の定義
def objective(trial):
    # x, y の範囲を定義
    x = trial.suggest_float("x", -5.0, 5.0)
    y = trial.suggest_float("y", -5.0, 5.0)
    
    # 二次関数 (例えば (x-2)^2 + (y+3)^2 のようなもの)
    return (x - 2) ** 2 + (y + 3) ** 2

# ベイズ最適化を使用して目的関数を最小化
def main():
    # BoTorchSampler を使用
    sampler = BoTorchSampler(
        candidates_func=matern52_logei_candidates_func,  # カスタム関数を指定
        n_startup_trials=10,
        seed=42
    )
    
    # OptunaのStudyを作成
    study = optuna.create_study(direction="minimize", sampler=sampler)
    
    # 最適化の実行
    study.optimize(objective, n_trials=50)
    
    # 結果の出力
    print("Best Value: ", study.best_value)
    print("Best Parameters: ", study.best_params)

if __name__ == "__main__":
    main()

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_26738/3909582317.py:17: ExperimentalWarning: BoTorchSampler is experimental (supported from v2.4.0). The interface can change in the future.
  sampler = BoTorchSampler(
[I 2024-11-26 15:13:48,610] A new study created in memory with name: no-name-6b6cd7b4-1701-472e-9c36-1108f045b627
[I 2024-11-26 15:13:48,615] Trial 0 finished with value: 66.94961040884104 and parameters: {'x': -1.254598811526375, 'y': 4.507143064099162}. Best is trial 0 with value: 66.94961040884104.
[I 2024-11-26 15:13:48,617] Trial 1 finished with value: 15.995219933491047 and parameters: {'x': 2.3199394181140507, 'y': 0.986584841970366}. Best is trial 1 with value: 15.995219933491047.
[I 2024-11-26 15:13:48,618] Trial 2 finished with value: 29.78522017865361 and parameters: {'x': -3.439813595575635, 'y': -3.4400547966379733}. Best is trial 1 with value: 15.995219933491047.
[I 2024-11-26 15:13:48,620] Trial 3 finished with value: 85.58473061665742 and paramet

Best Value:  0.0009877398691478618
Best Parameters:  {'x': 1.9905893357930884, 'y': -2.9700136819810665}
